##1 Lista de librerias

In [ ]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 1.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##2 Funciones

In [ ]:
%run '/content/drive/MyDrive/Colab Notebooks/Tesis_Functions.ipynb'

balancear_datos Loaded
Selectkbest_features Loaded
rfe_features Loaded
Integración de Variables Loaded
evaluacion de los modelos Loaded
Pca_components Loaded
Improtancia con XGBoost y Random forest Loaded
Random Forest Loaded
quitar_tildes Loaded
completar_datos_serie_tiempo Loaded
eliminar_filas_nan Loaded
cambiar_numeros_menores Loaded
normalize_variables Loaded
calcular_crecimiento_ingresos Loaded
obtener_variables_correlacionadas Loaded
generar_componentes_principales Loaded
rfe_with_elasticnet Loaded
calcular_crecimiento_promedio_por_categoria_Predial Loaded
categorizar_por_boxplot Loaded
calcular_crecimiento_promedio_por_categoria Loaded
limitar_atipicos Loaded
crear_dummies Loaded


#3 Carga de Datos

In [ ]:
## 1. regresión--------------------------------------------------------------------------------------------------------------
# Recomendación del Modelo XGBoot de Regresión
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Recomendacion_XGBoost_Regresion.csv'
Reg_Recomenacion=pd.read_csv(ruta_archivo)
# Variables importantes al pronosticar con XGBoost Regresión
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Variables_Importantes_XGBoost_Regresion.csv'
Var_Reg_XGB=pd.read_csv(ruta_archivo)
# Variables seleccionadas con reduccion de dimensionalidad de Regresión
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Variables_Importantes_selector_Regresion.csv'
Var_Reg_PCA=pd.read_csv(ruta_archivo)

## 2.Calasificación--------------------------------------------------------------------------------------------------------------
# Recomendación del Modelo XGBoot de Clasificación
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Output_Final_casificacion.csv'
Reg_Clasifiacion=pd.read_csv(ruta_archivo)
# Variables importantes al pronosticar con XGBoost Regresión
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Variables_Importantes_XGBoost_Classificacion.csv'
Var_Cla_XGB=pd.read_csv(ruta_archivo)
# Variables seleccionadas con reduccion de dimensionalidad de Regresión
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Variables_Importantes_selector_Clasificacion.csv'
Var_Cla_PCA=pd.read_csv(ruta_archivo)

In [ ]:
#Lista de municipios (Regresión: ok; Clasifficación: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=1a9IfZDuiqbhh2iefH8BMarHL8IXWpayF'
df = pd.read_csv(archivo_csv, sep=';', encoding='latin-1')
Municipios = pd.read_csv(archivo_csv, sep=';', encoding='latin-1')

In [ ]:
#Base_0_clasificacion
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Base_0_Variables.csv'
df_unido_Normalizada_class=pd.read_csv(ruta_archivo)
df_unido_Normalizada_class=df_unido_Normalizada_class.fillna(0)
Var_col=['Nombre_Municipio','Id_Departamento_Dane','Nombre_Departamento']
df_unido_Normalizada_class.drop(columns=Var_col, inplace=True)
df_unido_Normalizada_class['Poblacion_TotalHogares'] = df_unido_Normalizada_class['Poblacion_TotalHogares'].replace(0, 118)
Var_col=['Poblacion_TotalHogares','Id_Municipio_Dane']
df_var=df_unido_Normalizada_class.copy()
df_var.drop(columns=Var_col, inplace=True)
columnas_numericas=df_var.columns
for Columna in columnas_numericas:
    df_unido_Normalizada_class[Columna]=df_unido_Normalizada_class[Columna]/df_unido_Normalizada_class['Poblacion_TotalHogares']


In [ ]:
#Impuesto predial por municipio (Regresión: ok)
archivo_csv = 'https://drive.google.com/uc?export=download&id=15tKplCTM0AV7HS5tTBaaQdWBPBm3sX5J'
Predial_Final = pd.read_csv(archivo_csv, sep=';',thousands=',', decimal='.')
Predial_Final['Fecha']=pd.to_datetime(Predial_Final['Year'], format='%Y')
#quita atipicos con boxplot
Predial_Final_sin_atipicos=normalizar_serie_tiempo_por_categoria(Predial_Final, 'Fecha', 'Municipio', 'Predial')
Predial_Final_sin_atipicos['Predial'] = Predial_Final_sin_atipicos['Predial'].replace(0, np.nan)

#Generar los ajustes de la variables de valroes sin atipicos

#funcion de ajuste de valores sin atipicos----------------------------------------------------------------------------------------------------------------------------
funciones = [funcion_lineal, funcion_exponencial, funcion_polinomica, funcion_logaritmica,
             funcion_coseno, funcion_seno, funcion_cuadratica, funcion_cubica,
             funcion_raiz_cuadrada, funcion_logistica]
df=Predial_Final_sin_atipicos
df=df.fillna(0)
df_predial=ajustar_funcion2(df,'Fecha','Predial','Municipio',funciones,tolerancia_porcentaje=50)
df_predial.drop(columns=['Predial','Anio','Fecha'], inplace=True)
df_predial.rename(columns={'Valores_Ajustados': 'Predial','Municipio': 'CODIGO_DANE','Year':'Año'}, inplace=True)
Predial_Transpuesto=df_predial
Predial_Transpuesto=reemplazar_valores_negativos(Predial_Transpuesto, 'CODIGO_DANE', 'Predial')

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
<ipython-input-6-4a8a1804ba47>:11: RuntimeWarning: overflow encountered in exp
  return a * np.exp(b * x)
<ipython-input-6-4a8a1804ba47>:11: RuntimeWarning: overflow encountered in exp
  return a * np.exp(b * x)
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
<ipython-input-6-4a8a1804ba47>:106: RuntimeWarning: divide by zero encountered in true_divide
  diferencia_porcentaje = 100 * np.abs((valores_categoria - valores_ajustados) / valores_categoria)


In [ ]:
Predial_Transpuesto.head()

,CODIGO_DANE,Año,Predial
0,5001,2008,2.922120e+11
1,5001,2009,2.922120e+11
2,5001,2010,3.611887e+11
3,5001,2011,4.107830e+11
4,5001,2012,3.760985e+11


# 4 Crear factores por municipio

### 4.1. Eje Y, tamaño de ingreso

In [ ]:
Valor_Eje_Y = Predial_Transpuesto [Predial_Transpuesto ['Año'].isin([2017,2018,2019])]
Valor_Eje_Y=Valor_Eje_Y[['CODIGO_DANE','Predial']].groupby('CODIGO_DANE').mean()
Valor_Eje_Y=Valor_Eje_Y.reset_index()
Valor_Eje_Y['Predial_Ordenado']=Valor_Eje_Y['Predial'].rank(ascending=True).astype(int)

#Normalizar entre 1 y -1
data = np.array(Valor_Eje_Y['Predial_Ordenado'])
data_min = data.min()
data_max = data.max()
data_range = data_max - data_min
normalized_data = ((data - data_min) / data_range) * 2 - 1
Valor_Eje_Y['Predial_Eje_Y']=normalized_data
Valor_Eje_Y.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,CODIGO_DANE,Predial,Predial_Ordenado,Predial_Eje_Y
0,5001,6.983258e+11,1101,0.998183
1,5002,7.292926e+08,735,0.333333
2,5004,5.524962e+07,119,-0.785649
3,5021,2.059116e+08,412,-0.253406
4,5030,1.929583e+09,909,0.649410


### 4.2. Factor de crecimiento

In [ ]:
Regresion_value=Reg_Recomenacion.groupby('Municipio', as_index=False)['y_pred'].mean()

In [ ]:
df_Potencial=Municipios.merge(Regresion_value, left_on='Id_Municipio_Dane', right_on='Municipio',how='inner')
df_Potencial=df_Potencial.merge(Reg_Clasifiacion,on='Id_Municipio_Dane',how='inner')
Columans_Borrar=['Municipio','ID','y','recomendacion','Y_Objetivo']
df_Potencial=df_Potencial.drop(columns=Columans_Borrar)
df_Potencial['Score_X']=df_Potencial['y_pred']*10+df_Potencial['Score_1']+df_Potencial['Score_2']*2+df_Potencial['Score_0']*4+df_Potencial['Score_3']*6
df_Potencial['Score_X']=df_Potencial['Score_X'].rank(ascending=True).astype(int)

#Normalizar entre 1 y -1
data = np.array(df_Potencial['Score_X'])
data_min = data.min()
data_max = data.max()
data_range = data_max - data_min
normalized_data = ((data - data_min) / data_range) * 2 - 1
df_Potencial['Predial_Eje_X']=normalized_data
df_Potencial.head()


,Id_Municipio_Dane,Nombre_Municipio,Id_Departamento_Dane,Nombre_Departamento,y_pred,Score_0,Score_1,Score_2,Score_3,Score_X,Predial_Eje_X
0,5001,Medellín,5,Antioquia,0.596368,0.004794,0.002492,0.992076,0.000639,612,0.110909
1,5002,Abejorral,5,Antioquia,0.454807,0.001294,0.033988,0.964394,0.000325,419,-0.240000
2,5004,Abriaquí,5,Antioquia,0.515902,0.003939,0.018654,0.975592,0.001816,504,-0.085455
3,5021,Alejandría,5,Antioquia,0.819404,0.264146,0.006673,0.718947,0.010234,908,0.649091
4,5030,Amagá,5,Antioquia,0.724268,0.010996,0.001220,0.982140,0.005643,751,0.363636


# 5 Definicion de categoria

In [ ]:
def asignar_categoria(row):
    if row['Predial_Eje_X'] > 0 and row['Predial_Eje_Y'] > 0 :
        return 'Grandes Creciendo'
    elif row['Predial_Eje_Y'] > 0 and row['Predial_Eje_X'] <= 0 :
        return 'Desaceledaro'
    elif row['Predial_Eje_Y'] <= 0 and row['Predial_Eje_X'] > 0.5 :
        return 'Esperanza Futura'
    elif row['Predial_Eje_Y'] <= 0 and row['Predial_Eje_X'] > 0 :
        return 'Emergentes'
    else:
        return 'Estaticos'

In [ ]:
Matriz_Potencial=df_Potencial.merge(Valor_Eje_Y, left_on='Id_Municipio_Dane', right_on='CODIGO_DANE',how='inner')
Matriz_Potencial=Matriz_Potencial[['Id_Municipio_Dane','Nombre_Municipio','Id_Departamento_Dane','Nombre_Departamento','Predial_Eje_X','Predial_Eje_Y']]
Matriz_Potencial['Categoria_Potencial']=Matriz_Potencial.apply(asignar_categoria, axis=1)
Matriz_Potencial.head()

,Id_Municipio_Dane,Nombre_Municipio,Id_Departamento_Dane,Nombre_Departamento,Predial_Eje_X,Predial_Eje_Y,Categoria_Potencial
0,5001,Medellín,5,Antioquia,0.110909,0.998183,Grandes Creciendo
1,5002,Abejorral,5,Antioquia,-0.240000,0.333333,Desaceledaro
2,5004,Abriaquí,5,Antioquia,-0.085455,-0.785649,Estaticos
3,5021,Alejandría,5,Antioquia,0.649091,-0.253406,Esperanza Futura
4,5030,Amagá,5,Antioquia,0.363636,0.649410,Grandes Creciendo


In [ ]:
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Matriz_Potencial.csv'
Matriz_Potencial.to_csv(ruta_archivo, index=False)

# 6 Variables representarivas por categoria

In [ ]:
Matriz_Potencial_X=Matriz_Potencial[['Id_Municipio_Dane','Categoria_Potencial']]

In [ ]:
Matriz_Potencial_X=Matriz_Potencial_X.merge(df_unido_Normalizada_class, on='Id_Municipio_Dane',how='inner')

In [ ]:
columnas_numericas=Matriz_Potencial_X.columns
# Obtener una lista de índices de los registros que deseas eliminar
indices_a_eliminar = [0,1,2,3,4,5,6]

# Eliminar los registros de tipo object utilizando numpy.delete()
columnas_numericas = np.delete(columnas_numericas, indices_a_eliminar)


In [ ]:
Matriz_Potencial_X=limitar_atipicos(Matriz_Potencial_X, columnas_numericas)

In [ ]:
# Lista con los nombres de las columnas numéricas a analizar
warnings.filterwarnings("ignore")
#columnas_numericas = ['IPS_Nivel_1']
columna_categoria='Categoria_Potencial'
# Crear un DataFrame vacío para almacenar los resultados
resultados_df = pd.DataFrame(index=Matriz_Potencial_X[columna_categoria].unique())

# Iterar para cada columna numérica y agregar los resultados como columnas en el DataFrame de resultados
for columna in columnas_numericas:
    df_temp=Matriz_Potencial_X[[columna_categoria,columna]]

    poblacion=Matriz_Potencial_X[[columna]]
    resultados_efecto = efecto_por_categoria(df_temp, columna_categoria, columna, poblacion, nivel_confianza=0.95)
    # Agregar los resultados como una nueva columna en el DataFrame de resultados
    #resultados_df[columna + '_Efecto'] = resultados_efecto[columna]
    resultados_df = pd.concat([resultados_df, resultados_efecto.add_suffix('_Efecto')], axis=1)

# Transponer el DataFrame
datos_transpuesto = resultados_df.transpose()

# Reiniciar el índice para que los valores numéricos se conviertan en una columna
datos_transpuesto = datos_transpuesto.reset_index()

# Cambiar el nombre de la columna del índice por 'Columna'
datos_transpuesto = datos_transpuesto.rename(columns={'index': 'Columna'})
warnings.filterwarnings("default")



In [ ]:
datos_transpuesto.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Columna,Grandes Creciendo,Desaceledaro,Estaticos,Esperanza Futura,Emergentes
0,IPS_Total_Efecto,No significativo,No significativo,No significativo,No significativo,No significativo
1,IPS_Departamental_Efecto,No significativo,No significativo,No significativo,No significativo,No significativo
2,IPS_Distrital_Efecto,No significativo,No significativo,No significativo,No significativo,No significativo
3,IPS_Indigena_Efecto,No significativo,No significativo,No significativo,No significativo,No significativo
4,IPS_Municipal_Efecto,No significativo,No significativo,No significativo,No significativo,No significativo


In [ ]:
ruta_archivo = '/content/drive/My Drive/4_Proyect_Final/1_Datos/Variables_categorias.csv'
datos_transpuesto.to_csv(ruta_archivo, index=False)